# Introduction to SparkSQL

## Spark Initiallization Script

In [2]:
#!/usr/bin/env python3

import sys
import os 

# getting the directory where Spark was installed
if 'SPARK_HOME' not in os.environ:
    os.environ['SPARK_HOME'] = '/opt/spark'

# python variable to store the root path for later reference
SPARK_HOME = os.environ['SPARK_HOME']

# adding pyspark and py4j packages paths to python path env variable
sys.path.insert(0, os.path.join(SPARK_HOME, 'python'))
sys.path.insert(0, os.path.join(SPARK_HOME, 'python', 'lib'))
sys.path.insert(0, os.path.join(SPARK_HOME, 'python', 'lib', 'py4j-0.10.9-src.zip'))
sys.path.insert(0, os.path.join(SPARK_HOME, 'python', 'lib', 'pyspark.zip'))

from pyspark import SparkContext
from pyspark import SparkConf

# configure spark settings
conf = SparkConf()
conf.set("spark.executer.memory","1g")
conf.set("spark.cores.max",'2')

# give name to your spark application
conf.setAppName("IntroSparkSql")

# create a spark context object 
# note: Execute only once otherwise results in Context Errors
sc = SparkContext('local',conf=conf)

### Once the above script is executed you can view the Spark instance info here http://localhost:4040

### Creating SQL context from Spark context

In [3]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

### Creating Dataframe from json file

In [4]:
empDf = sqlContext.read.json("data/customerData.json")
empDf.show()

+---+------+------+-----------------+------+
|age|deptid|gender|             name|salary|
+---+------+------+-----------------+------+
| 32|   100|  male|Benjamin Garrison|  3000|
| 40|   200|  male|    Holland Drake|  4500|
| 26|   100|  male|  Burks Velasquez|  2700|
| 51|   100|female|    June Rutledge|  4300|
| 44|   200|  male|    Nielsen Knapp|  6500|
+---+------+------+-----------------+------+



schema of dataframe

In [5]:
empDf.printSchema()

root
 |-- age: string (nullable = true)
 |-- deptid: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: string (nullable = true)



### Performing Queries on the dataframe

In [6]:
 empDf.select("name").show()

+-----------------+
|             name|
+-----------------+
|Benjamin Garrison|
|    Holland Drake|
|  Burks Velasquez|
|    June Rutledge|
|    Nielsen Knapp|
+-----------------+



In [7]:
empDf.select(empDf.salary,empDf.deptid).show()

+------+------+
|salary|deptid|
+------+------+
|  3000|   100|
|  4500|   200|
|  2700|   100|
|  4300|   100|
|  6500|   200|
+------+------+



In [8]:
empDf.filter(empDf["age"]==40).show()

+---+------+------+-------------+------+
|age|deptid|gender|         name|salary|
+---+------+------+-------------+------+
| 40|   200|  male|Holland Drake|  4500|
+---+------+------+-------------+------+



In [9]:
empDf.groupBy("gender").count().show()

+------+-----+
|gender|count|
+------+-----+
|female|    1|
|  male|    4|
+------+-----+



In [10]:
empDf.groupBy("deptid").agg({"salary":"avg","age":"max"}).show()

+------+------------------+--------+
|deptid|       avg(salary)|max(age)|
+------+------------------+--------+
|   200|            5500.0|      44|
|   100|3333.3333333333335|      51|
+------+------------------+--------+



### Creating dataframe from list

In [11]:
deptList = [{"name":"sales","id":"100"},{"name":"engineering","id":"200"}]
deptDf = sqlContext.createDataFrame(deptList)
deptDf.show()

+---+-----------+
| id|       name|
+---+-----------+
|100|      sales|
|200|engineering|
+---+-----------+



### Using Join on dataframes

In [12]:
empDf.join(deptDf,empDf.deptid == deptDf.id).show()

+---+------+------+-----------------+------+---+-----------+
|age|deptid|gender|             name|salary| id|       name|
+---+------+------+-----------------+------+---+-----------+
| 51|   100|female|    June Rutledge|  4300|100|      sales|
| 26|   100|  male|  Burks Velasquez|  2700|100|      sales|
| 32|   100|  male|Benjamin Garrison|  3000|100|      sales|
| 44|   200|  male|    Nielsen Knapp|  6500|200|engineering|
| 40|   200|  male|    Holland Drake|  4500|200|engineering|
+---+------+------+-----------------+------+---+-----------+



### Cascading Operations

In [13]:
empDf.filter(empDf.age > 30) \
    .join(deptDf,deptDf.id == empDf.deptid) \
    .groupBy(empDf.deptid).agg({"salary":"avg","age":"max"}).show()

+------+-----------+--------+
|deptid|avg(salary)|max(age)|
+------+-----------+--------+
|   200|     5500.0|      44|
|   100|     3650.0|      51|
+------+-----------+--------+



### Registering dataframes as tables and run SQL queries

In [14]:
empDf.registerTempTable("employees")
sqlContext.sql("SELECT * FROM `employees` WHERE `salary` > 4000;").show()

+---+------+------+-------------+------+
|age|deptid|gender|         name|salary|
+---+------+------+-------------+------+
| 40|   200|  male|Holland Drake|  4500|
| 51|   100|female|June Rutledge|  4300|
| 44|   200|  male|Nielsen Knapp|  6500|
+---+------+------+-------------+------+



In [15]:
sqlContext.sql("SELECT * FROM `employees` WHERE name LIKE '%Drake%' LIMIT 1;").show()

+---+------+------+-------------+------+
|age|deptid|gender|         name|salary|
+---+------+------+-------------+------+
| 40|   200|  male|Holland Drake|  4500|
+---+------+------+-------------+------+



### Converting spark dataframe to pandas dataframe

In [16]:
import pandas as pd
empPands = empDf.toPandas()
empPands

,age,deptid,gender,name,salary
0,32,100,male,Benjamin Garrison,3000
1,40,200,male,Holland Drake,4500
2,26,100,male,Burks Velasquez,2700
3,51,100,female,June Rutledge,4300
4,44,200,male,Nielsen Knapp,6500


In [17]:
for index, row in empPands.iterrows():
    print (row['salary'])

3000
4500
2700
4300
6500


In [19]:

dfMySql = sqlContext.read.format("jdbc").options(
    url="jdbc:mysql://localhost:3306/socialem",
    driver = "com.mysql.jdbc.Driver",
    dbtable = "users",
    user="root",
    password="root").load()
dfMySql.show()

+---+--------------+--------------------+------------+-----------------+--------------------+--------------+-------------------+-------------------+
| id|          name|               email|home_page_id|email_verified_at|            password|remember_token|         created_at|         updated_at|
+---+--------------+--------------------+------------+-----------------+--------------------+--------------+-------------------+-------------------+
|  1|Muhammad Ateeb|muhammadateeb225@...|           1|             null|$2y$10$NMw14A2.n4...|          null|2021-01-29 07:19:49|2021-01-29 07:19:49|
|  2|         Eteeb|muhammadateeb226@...|           2|             null|$2y$10$l3S4TtsY5d...|          null|2021-03-11 07:24:58|2021-03-11 07:24:58|
+---+--------------+--------------------+------------+-----------------+--------------------+--------------+-------------------+-------------------+



pyspark.sql.dataframe.DataFrame

In [20]:
dfMySql.filter(dfMySql.home_page_id == 1).show()

+---+--------------+--------------------+------------+-----------------+--------------------+--------------+-------------------+-------------------+
| id|          name|               email|home_page_id|email_verified_at|            password|remember_token|         created_at|         updated_at|
+---+--------------+--------------------+------------+-----------------+--------------------+--------------+-------------------+-------------------+
|  1|Muhammad Ateeb|muhammadateeb225@...|           1|             null|$2y$10$NMw14A2.n4...|          null|2021-01-29 07:19:49|2021-01-29 07:19:49|
+---+--------------+--------------------+------------+-----------------+--------------------+--------------+-------------------+-------------------+



#### Inorder to connect to the database you need to have the ```com.mysql.jdbc.Driver``` reachable by spark application, you can acomplish this by simply downloading the driver and saving it to the ```$SPARK_HOME/python/lib``` directory and restart the kernel for the changes to take effect

In [48]:
tables = sqlContext.read.format("jdbc").options(
    url="jdbc:mysql://localhost:3306/socialem",
    driver = "com.mysql.jdbc.Driver",
    query = "SELECT * FROM migrations",
    user="root",
    password="root").load()
tables.show()

+---+--------------------+-----+
| id|           migration|batch|
+---+--------------------+-----+
|  1|2014_10_12_000000...|    1|
|  2|2014_10_12_100000...|    1|
|  3|2019_08_19_000000...|    1|
|  4|2021_01_16_183952...|    1|
|  5|2021_01_17_062341...|    1|
|  6|2021_01_17_062401...|    1|
|  7|2021_01_17_062411...|    1|
|  8|2021_01_17_062422...|    1|
|  9|2021_01_17_062431...|    1|
| 10|2021_01_17_062440...|    1|
| 11|2021_01_28_172632...|    1|
+---+--------------------+-----+



### Creating Dataframe from RDDs

In [71]:
from pyspark.sql import Row

lines = sc.textFile("data/auto-data.csv")

# remove the header line
datalines = lines.filter(lambda l: "FUELTYPE" not in l)
datalines.count()

197

In [76]:
parts = datalines.map(lambda l: l.split(","))
parts.take(1)

[['subaru',
  'gas',
  'std',
  'two',
  'hatchback',
  'fwd',
  'four',
  '69',
  '4900',
  '31',
  '36',
  '5118']]

In [50]:
autoMap = parts.map(lambda p: Row(make=p[0], body=p[4], hp=int(p[7])))
autoMap.take(5)

[Row(make='subaru', body='hatchback', hp=69),
 Row(make='chevrolet', body='hatchback', hp=48),
 Row(make='mazda', body='hatchback', hp=68),
 Row(make='toyota', body='hatchback', hp=62),
 Row(make='mitsubishi', body='hatchback', hp=68)]

### Infer Schema and register dataframe as a table

In [51]:
autoDf = sqlContext.createDataFrame(autoMap)
autoDf.head(10)

[Row(make='subaru', body='hatchback', hp=69),
 Row(make='chevrolet', body='hatchback', hp=48),
 Row(make='mazda', body='hatchback', hp=68),
 Row(make='toyota', body='hatchback', hp=62),
 Row(make='mitsubishi', body='hatchback', hp=68),
 Row(make='honda', body='hatchback', hp=60),
 Row(make='nissan', body='sedan', hp=69),
 Row(make='dodge', body='hatchback', hp=68),
 Row(make='plymouth', body='hatchback', hp=68),
 Row(make='mazda', body='hatchback', hp=68)]

In [60]:
autoDf.registerTempTable("autos")
sqlContext.sql("SELECT * FROM autos WHERE hp > 180;").show()

+-------------+-----------+---+
|         make|       body| hp|
+-------------+-----------+---+
|       nissan|  hatchback|200|
|          bmw|      sedan|182|
|      porsche|    hardtop|207|
|      porsche|    hardtop|207|
|       jaguar|      sedan|262|
|          bmw|      sedan|182|
|      porsche|convertible|207|
|mercedes-benz|      sedan|184|
|          bmw|      sedan|182|
|mercedes-benz|    hardtop|184|
+-------------+-----------+---+



In [61]:
dfAutoPandas = autoDf.toPandas()
dfAutoPandas.head()

,make,body,hp
0,subaru,hatchback,69
1,chevrolet,hatchback,48
2,mazda,hatchback,68
3,toyota,hatchback,62
4,mitsubishi,hatchback,68


In [84]:
dfAutoPandas.groupby(['body','make']).mean()

hp
body        make                     
convertible alfa-romero    111.000000
            mercedes-benz  155.000000
            porsche        207.000000
            toyota         116.000000
            volkswagen      90.000000
hardtop     mercedes-benz  153.500000
            nissan          69.000000
            porsche        207.000000
            toyota         116.000000
hatchback   alfa-romero    154.000000
            chevrolet       59.000000
            dodge           90.200000
            honda           74.000000
            isuzu           90.000000
            mazda           89.400000
            mercury        175.000000
            mitsubishi     105.000000
            nissan         137.200000
            plymouth        95.750000
            porsche        143.000000
            saab           126.666667
            subaru          71.666667
            toyota          93.000000
            volkswagen      90.000000
sedan       audi           115.400000
            bmw            138.875000
            chevrolet       70.000000
            dodge           68.000000
            honda           89.800000
            isuzu           78.000000
            jaguar         204.666667
            mazda           82.666667
            mercedes-benz  146.250000
            mitsubishi     102.000000
            nissan          89.000000
            peugot         102.285714
            plymouth        68.000000
            saab           126.666667
            subaru          90.200000
            toyota          86.100000
            volkswagen      78.333333
            volvo          127.250000
wagon       audi           110.000000
            dodge           88.000000
            honda           76.000000
            mercedes-benz  123.000000
            nissan          96.666667
            peugot          95.500000
            plymouth        88.000000
            subaru          92.250000
            toyota          85.500000
            volkswagen      88.000000
            volvo          130.000000

In [93]:
import numpy as np
table = pd.pivot_table(dfAutoPandas,index=['make'],columns=['body'],values=['hp'])
table

hp                                            
body          convertible hardtop   hatchback       sedan       wagon
make                                                                 
alfa-romero         111.0     NaN  154.000000         NaN         NaN
audi                  NaN     NaN         NaN  115.400000  110.000000
bmw                   NaN     NaN         NaN  138.875000         NaN
chevrolet             NaN     NaN   59.000000   70.000000         NaN
dodge                 NaN     NaN   90.200000   68.000000   88.000000
honda                 NaN     NaN   74.000000   89.800000   76.000000
isuzu                 NaN     NaN   90.000000   78.000000         NaN
jaguar                NaN     NaN         NaN  204.666667         NaN
mazda                 NaN     NaN   89.400000   82.666667         NaN
mercedes-benz       155.0   153.5         NaN  146.250000  123.000000
mercury               NaN     NaN  175.000000         NaN         NaN
mitsubishi            NaN     NaN  105.000000  102.000000         NaN
nissan                NaN    69.0  137.200000   89.000000   96.666667
peugot                NaN     NaN         NaN  102.285714   95.500000
plymouth              NaN     NaN   95.750000   68.000000   88.000000
porsche             207.0   207.0  143.000000         NaN         NaN
saab                  NaN     NaN  126.666667  126.666667         NaN
subaru                NaN     NaN   71.666667   90.200000   92.250000
toyota              116.0   116.0   93.000000   86.100000   85.500000
volkswagen           90.0     NaN   90.000000   78.333333   88.000000
volvo                 NaN     NaN         NaN  127.250000  130.000000